In [1]:
import numpy as np
import pandas as pd

In [2]:
Promo_sales_data=pd.read_csv(r"C:\Users\ss\OneDrive\Desktop\Msc Project\Datasets\Promo_sales_data.csv")
Promo_sales_data['DATE'] = pd.to_datetime(Promo_sales_data['DATE'])
Promo_sales_data

,DATE,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES,TOTAL SALES
0,2017-06-01,2017,6,BARON FRANCOIS LTD,438790,RAISSAC LES LYS RSE DE CAB - 750ML,WINE,19.34,15.0,11.0,30.34
1,2017-06-01,2017,6,E & J GALLO WINERY,439010,LIBERTY CREEK SWEET RED - 1.5L,WINE,9.56,7.0,25.0,34.56
2,2017-06-01,2017,6,CONSTELLATION BRANDS,439355,BLUFELD MEDIUM SWEET RIESLING - 750ML,WINE,16.73,10.0,15.0,31.73
3,2017-06-01,2017,6,DIONYSOS IMPORTS INC,4421,SUPERBOCK BEER 4/6NR - 11.2OZ,BEER,0.00,0.0,30.0,30.00
4,2017-06-01,2017,6,CONSTELLATION BRANDS,44039,SVEDKA VODKA - 1L,LIQUOR,25.71,36.0,6.0,31.71
...,...,...,...,...,...,...,...,...,...,...,...
44786,2020-09-01,2020,9,PERNOD RICARD USA LLC,32140,BEEFEATER GIN - 1.75L,LIQUOR,68.02,62.0,0.0,68.02
44787,2020-09-01,2020,9,JACKSON FAMILY ENTERPRISES INC,307622,MURPHY GOODE P/NOIR - 750ML,WINE,16.80,19.0,23.0,39.80
44788,2020-09-01,2020,9,YUENGLING BREWERY,3077,YUENGLING BLACK & TAN 2/12 NR - 12OZ,BEER,27.00,19.0,167.0,194.00
44789,2020-09-01,2020,9,DUCKHORN WINE COMPANY,321828,DECOY CHARD - 750ML,WINE,15.26,16.0,14.0,29.26


In [3]:
Promo_sales_data.dtypes

DATE                datetime64[ns]
YEAR                         int64
MONTH                        int64
SUPPLIER                    object
ITEM CODE                   object
ITEM DESCRIPTION            object
ITEM TYPE                   object
RETAIL SALES               float64
RETAIL TRANSFERS           float64
WAREHOUSE SALES            float64
TOTAL SALES                float64
dtype: object

In [4]:
Promo_sales_data['YEAR'] = Promo_sales_data['DATE'].dt.year
Promo_sales_data['MONTH'] = Promo_sales_data['DATE'].dt.month
Promo_sales_data['QUARTER'] = Promo_sales_data['DATE'].dt.quarter



In [5]:
Promo_sales_data.nunique()

DATE                   24
YEAR                    4
MONTH                  12
SUPPLIER              205
ITEM CODE            4153
ITEM DESCRIPTION     4436
ITEM TYPE               7
RETAIL SALES        10566
RETAIL TRANSFERS     2252
WAREHOUSE SALES      4637
TOTAL SALES         15983
QUARTER                 4
dtype: int64

In [6]:
# Example for 1-month lag
Promo_sales_data['SALES_LAG_1'] = Promo_sales_data.groupby('ITEM DESCRIPTION')['TOTAL SALES'].shift(1)
Promo_sales_data['SALES_LAG_3'] = Promo_sales_data.groupby('ITEM DESCRIPTION')['TOTAL SALES'].shift(3)
Promo_sales_data['SALES_LAG_6'] = Promo_sales_data.groupby('ITEM DESCRIPTION')['TOTAL SALES'].shift(6)

# Example rolling average over last 3 months
Promo_sales_data['SALES_ROLLING_MEAN_3'] = Promo_sales_data.groupby('ITEM DESCRIPTION')['TOTAL SALES'].transform(lambda x: x.rolling(window=3).mean())


In [7]:
# One-hot encoding for ITEM TYPE
Promo_sales_data = pd.get_dummies(Promo_sales_data, columns=['ITEM TYPE'], prefix='TYPE')

# Label encoding for SUPPLIER
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Promo_sales_data['SUPPLIER_ENCODED'] = le.fit_transform(Promo_sales_data['SUPPLIER'])


In [8]:
# Median sales of each product across all periods
median_sales = Promo_sales_data.groupby('ITEM DESCRIPTION')['TOTAL SALES'].median().reset_index()
median_sales.rename(columns={'TOTAL SALES': 'MEDIAN_PRODUCT_SALES'}, inplace=True)

# Merge back
Promo_sales_data = Promo_sales_data.merge(median_sales, on='ITEM DESCRIPTION', how='left')


In [9]:
# Data Structure
normal_data_unstack = Promo_sales_data.groupby(['YEAR', 'MONTH']).size().unstack(fill_value=0)
normal_data_unstack

MONTH,1,2,3,4,5,6,7,8,9,10,11,12
YEAR,,,,,,,,,,,,
2017,0,0,0,0,0,1920,1806,1840,1792,1819,2008,2054
2018,1606,1693,0,0,0,0,0,0,0,0,0,0
2019,1714,1748,1845,1825,2024,1896,1964,1919,1781,1992,1978,0
2020,1800,0,2034,0,0,0,1897,0,1836,0,0,0


Promo_sales_data.to_csv('Promo_sales_data_with_features.csv', index=False)
